In [1]:
!pip install dwave-system==1.18.0
!pip install dwave-system[drivers] --extra-index-url https://pypi.dwavesys.com/simple
!pip install dimod==0.11.0
!pip install dwave_qbsolv

!pip install dwave-ocean-sdk>=6.3.0
!pip install mip==1.13.0
!pip install plotly==5.6.0
!pip install streamlit==1.9.2
!pip install tabulate==0.8.9


  Using cached dimod-0.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.7 MB)
  Attempting uninstall: dimod
    Found existing installation: dimod 0.12.4
    Uninstalling dimod-0.12.4:
      Successfully uninstalled dimod-0.12.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dwave-system 1.18.0 requires dimod<0.14.0,>=0.12.0, but you have dimod 0.11.0 which is incompatible.
dwave-samplers 1.0.0 requires dimod<0.13.0,>=0.12.0, but you have dimod 0.11.0 which is incompatible.
dwave-qbsolv 0.3.4 requires dimod<0.11.0,>=0.8.1, but you have dimod 0.11.0 which is incompatible.
dwave-preprocessing 0.5.4 requires dimod<0.13.0,>=0.12.2, but you have dimod 0.11.0 which is incompatible.
  Using cached dimod-0.10.17-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (14.1 MB)
  Attempting uninstall: dimod
    Found existing installation: dimod 0

In [2]:
from dwave_qbsolv import QBSolv
from dwave.system import DWaveSampler


In [ ]:
!pip index versions dimod

In [ ]:
dimod.__version__